### Introducing the xUnit framework

- You can install the `C# Dev Kit` extension for VS Code to use the GUI for testing
    * You must build the test project for the tests to appear in the Test Explorer
- Creating a xUnit project `MyProject.Tests`


    ```bash
    dotnet new xunit
    ```

    Add a reference to the project you are testing.

    ```bash
    dotnet add reference ../../src/MyProject.csproj.
    ```

- `[fact]` attribute is used to define a test case
- `[Theroy]` attribute is used to define a data driven test case
- Running tests with the CLI

    ```
    dotnet test
    ```

In [3]:
#r "nuget:xunit"
#r "nuget:xunit.v3.runner.utility"

Installed Packages xunit, 2.9.3 xunit.v3.runner.utility, 2.0.1

A simple test runner with help from Copilot.

In [50]:
using System;
using System.Linq;
using System.Reflection;
using Xunit;

public class SimpleTestRunner
{
    public static void RunTests(Type testClassType)
    {
        Console.WriteLine($"Running tests in {testClassType.Name}...");

        // Create an instance of the test class
        var testClassInstance = Activator.CreateInstance(testClassType);

        // Get all methods with the [Fact] or [Theory] attribute
        var testMethods = testClassType.GetMethods()
            .Where(m => m.GetCustomAttributes(typeof(FactAttribute), false).Any() ||
                        m.GetCustomAttributes(typeof(TheoryAttribute), false).Any());

        foreach (var method in testMethods)
        {
            var theoryData = method.GetCustomAttributes(typeof(InlineDataAttribute), false)
                                   .Cast<InlineDataAttribute>();
            
            if (theoryData.Any())
            {
                // Handle [Theory] with [InlineData]
                foreach (var data in theoryData)
                {
                    try
                    {
                        
                        Object[] myObjArray = new Object[] {
                             data.GetData(method)
                        };
                        Console.WriteLine($"Running {method.Name} with arguments: {String.Join(",", data.GetData(method).ToArray<object[]>()[0])}...");
                        method.Invoke(testClassInstance, data.GetData(method).ToArray<object[]>()[0]);
                        Console.WriteLine($"✔ {method.Name} passed.");
                    }
                    catch (TargetInvocationException ex)
                    {
                        Console.WriteLine($"✘ {method.Name} failed: {ex.InnerException?.Message}");
                        if ( ex.InnerException?.GetType().Namespace != "Xunit.Sdk" )
                        {
                            Console.Write($"{ex.InnerException?.StackTrace}");
                        }
                    }
                }
            }
            else
            {
                // Handle [Fact]
                try
                {
                    Console.WriteLine($"Running {method.Name}...");
                    method.Invoke(testClassInstance, null);
                    Console.WriteLine($"✔ {method.Name} passed.");
                }
                catch (TargetInvocationException ex)
                {
                    Console.WriteLine($"✘ {method.Name} failed: {ex.InnerException?.Message}");
                    if ( ex.InnerException?.GetType().Namespace != "Xunit.Sdk" )
                    {
                        Console.Write($"{ex.InnerException?.StackTrace}");
                    }
                }
            }
        }
    }
}

In [5]:
using Xunit;

public class CalculatorTests
{
    [Fact]
    public void Add_ShouldReturnCorrectSum()
    {
        int result = Add(2, 3);
        Assert.Equal(5, result);
    }

    [Fact]
    public void Add_Failure()
    {
        int result = Add(1, 1);
        Assert.Equal(3, result);
    }
    [Fact]
    public void TestException()
    {
        throw new Exception("This test is intentionally failing.");
    }

    private int Add(int a, int b) => a + b;
}

In [17]:
SimpleTestRunner.RunTests(typeof(CalculatorTests));

Running tests in CalculatorTests...
Running Add_ShouldReturnCorrectSum...
✔ Add_ShouldReturnCorrectSum passed.
Running Add_Failure...
✘ Add_Failure failed: Assert.Equal() Failure: Values differ
Expected: 3
Actual:   2
Running TestException...
✘ TestException failed: This test is intentionally failing.
   at Submission#3.CalculatorTests.TestException()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodBaseInvoker.InvokeWithNoArgs(Object obj, BindingFlags invokeAttr)

Use the [Theory] attribute to test with inline data.

In [51]:
using Xunit;

public class InlineDataTest
{
    [Theory]
    [InlineData(1, 1)]
    [InlineData(2, 2)]
    [InlineData(5, 5)]
    public void Should_be_equal(int value1, int value2)
    {
        Assert.Equal(value1, value2);
    }
}
SimpleTestRunner.RunTests(typeof(InlineDataTest));

Running tests in InlineDataTest...
Running Should_be_equal with arguments: 1,1...
✔ Should_be_equal passed.
Running Should_be_equal with arguments: 2,2...
✔ Should_be_equal passed.
Running Should_be_equal with arguments: 5,5...
✔ Should_be_equal passed.
